### Извлекување на карактеристики од податоците на ниво на објект, месец (CustomerID, month). За секој објект и за секој месец се гради по еден вектор од карактеристики со следниве атрибути:
   - Број на планирани посети
   - Број на извршени планирани посети
   - Број на неизвршени планирани посети
   - Број на адхок посети
   - Денот (во месецот) на кој е направен најголем отстап во вредноста на SAPE во однос на претходната посета
   - Денот (во месецот) на кој е запишан најголем (max) SAPE коефициент
   - Денот (во месецот) на кој е запишан најмал (min) SAPE коефициент
   - Денот (во месецот) на кој е направена валидација (контрола) 
   - Број на валидации (посети од контролор)
   - Средна вредност за SAPE коефициенти
   - Стандардна девијација за SAPE коефициенти
   - Средна вредност за APE коефициенти
   - Стандардна девијација за APE коефициенти
   - Квадратна разлика помеѓу средните вредности за SAPE и APE коефициентите
   - Средна вредност за FPE коефициенти
   - Стандардна девијација за FPE коефициенти
   - Број на 'ok' валидации
   - Број на 'not ok' валидации
   - Процент на 'ok' валидации
   - Процент на 'not ok' валидации


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('dataset/Spenser_1_7_remove_v22_treshold.tsv',
                 delimiter='\t')

In [3]:
def get_dates_sapes(df_month):
    dates_sapes = []
    for date in list(df_month.Calendarday):
        dates_sapes.append((date, df_month[df_month.Calendarday == date].SAPECoeficient.values[0]))
    return dates_sapes


def get_first_not_nan(dates_sapes):
    for val in dates_sapes:
        if not np.isnan(val[1]):
            return val
    return None


def get_previous_not_nan_record(dates_sapes, i):
    if i-1 == 0:
        return dates_sapes[i-1][1]
    
    if not np.isnan(dates_sapes[i-1][1]):
        return dates_sapes[i-1][1]

    return get_previous_not_nan_record(dates_sapes, i-1)


def calculate_max_distance_day(df_month):
    max_index = 0
    max_distance = -float('inf')
    dates_sapes = get_dates_sapes(df_month)
    # dates_sapes = list(filter(lambda n: not np.isnan(n[1]), dates_sapes))
    
    if all(map(lambda tup: np.isnan(tup[1]), dates_sapes)):
        return np.nan  # there is no SAPE coefficient for this month

    first_not_nan = get_first_not_nan(dates_sapes)  # a (date, sape) record or None
    
    if all(map(lambda tup: tup[1] == first_not_nan[0] or np.isnan(tup[1]), dates_sapes)):
        return int(first_not_nan[1].split('-')[2])  #  all (not nan) sape coefficients are the same, return the date of the first one
    
    for i in range(1, len(dates_sapes) - 1):
        val_1 = dates_sapes[i][1]  
        val_2 = get_previous_not_nan_record(dates_sapes, i)
        
        if np.isnan(val_1) or np.isnan(val_2):
            continue
        distance = (abs(val_1 - val_2))
        if distance > max_distance:
            max_index = i
            max_distance = distance
            
    max_distance_date = dates_sapes[max_index][0]
    return int(max_distance_date.split('-')[2])


def calculate_day_of_maximum_sape(df_month):
    dates_sapes = get_dates_sapes(df_month)

    if all(map(lambda tup: np.isnan(tup[1]), dates_sapes)):
        return np.nan  # there is no SAPE coefficient for this month
    
    first_not_nan = get_first_not_nan(dates_sapes)

    if all(map(lambda tup: tup[1] == first_not_nan[1], dates_sapes)):
        return int(first_not_nan[0].split('-')[2])  # all (not nan) sape coefficients are the same, return the date of the first one

    max_index = 0
    max_sape = -float('inf')
    for i in range(len(dates_sapes)):
        val = dates_sapes[i]
        if val[1] > max_sape:
            max_index = i
            max_sape = val[1]

    return int(dates_sapes[max_index][0].split('-')[2])


def calculate_day_of_minimum_sape(df_month):
    dates_sapes = get_dates_sapes(df_month)

    if all(map(lambda tup: np.isnan(tup[1]), dates_sapes)):
        return np.nan  # there is no SAPE coefficient for this month
    
    first_not_nan = get_first_not_nan(dates_sapes)

    if all(map(lambda tup: tup[1] == first_not_nan[1], dates_sapes)):
        return int(first_not_nan[0].split('-')[2])  # all (not nan) sape coefficients are the same, return the date of the first one

    min_index = 0
    min_sape = float('inf')
    for i in range(len(dates_sapes)):
        val = dates_sapes[i]
        if val[1] < min_sape:
            min_index = i
            min_sape = val[1]

    return int(dates_sapes[min_index][0].split('-')[2])


def calculate_day_of_validation(df_month):
    df_temp = df_month[~ np.isnan(df_month.APECoeficient)]
    if len(df_temp) == 0:
        return np.nan  # no validations in this month, return nan
    else:
        return int(df_temp.Calendarday.values[0].split('-')[2])  # return the day of the only validation for this month


In [4]:
expert_vectors = dict()
unique_customers = set(df.CustomerID)

for customer in unique_customers:
    df_customer = df[df.CustomerID == customer]
    months = set(df_customer.month)
    for month in months:
        df_customer_month = df_customer[df_customer.month == month]

        if len(df_customer_month) == 0:
            continue

        no_planned_visits = len(
            df_customer_month[df_customer_month.PerformanceEvaluationPlanned == '1'])  # број на планирани посети
        no_executed_visits = len(
            df_customer_month[df_customer_month.PerformanceEvaluationExecuted == '1'])  # број на извршени планирани посети
        no_not_executed_visits = no_planned_visits - no_executed_visits  # број на неизвршени планирани посети
        no_ad_hoc = len(df_customer_month[df_customer_month.PerformanceEvaluationAdHoc == '1'])  # број на адхок посети

        day_of_maximum_difference = calculate_max_distance_day(
            df_customer_month)  # на кој ден (во месецот) е настаната најголема разлика во однос на претходна евалуација
        
        day_of_maximum_sape = calculate_day_of_maximum_sape(df_customer_month)  # денот (во месецот) на кој е запишан најголем SAPE коефициент
        day_of_mimimum_sape = calculate_day_of_minimum_sape(df_customer_month)  # денот (во месецот) на кој е запишан најмал SAPE коефициент
        
        day_of_validation = calculate_day_of_validation(df_customer_month)  # денот (во месецот) на кој е направена контрола
        
        mean_sape = np.mean(df_customer_month.SAPECoeficient)  # средна вредност за SAPE коефициенти
        std_sape = np.std(df_customer_month.SAPECoeficient)  # стандардна девијација за SAPE коефициенти

        mean_fpe = np.mean(df_customer_month.FPECoeficient)  # средна вредност за FPE коефициенти
        std_fpe = np.std(df_customer_month.FPECoeficient)  # стандардна девијација за FPE коефициенти

        mean_ape = np.mean(df_customer_month.APECoeficient)  # средна вредност за APE коефициенти
        # std_ape = np.std(df_customer_month.APECoeficient) # не постои девијација бидејќи има само една контрола

        squared_err_sape_ape = (mean_sape - mean_ape) ** 2  # квадратна разлика помеѓу средните вредности за SAPE и APE

        no_ok_validations = len(df_customer_month[df_customer_month.OK == 'ok'])  # број на 'ok'валидации (OK == 'ok')
        no_not_ok_validations = len(
            df_customer_month[df_customer_month.NOTOK == 'not ok'])  # број на 'not ok' валидации (NOTOK == 'not ok')

        no_validations = len(
            df_customer_month[~np.isnan(df_customer_month.APECoeficient)])  # вкупен број на валидации (посети од контролор)

        percent_not_ok = float(
            no_not_ok_validations) / no_validations if no_validations != 0 else 0  # процент на 'ok' валидации
        percent_ok = float(
            no_ok_validations) / no_validations if no_validations != 0 else 0  # процент на 'not ok' валидации

        expert_vectors[(customer, month)] = [customer, month, no_planned_visits, no_executed_visits, 
                                             no_not_executed_visits, no_ad_hoc, day_of_maximum_difference,
                                             day_of_maximum_sape, day_of_mimimum_sape, day_of_validation, 
                                             no_validations, mean_sape, std_sape, mean_ape, 
                                             squared_err_sape_ape, mean_fpe, std_fpe,
                                            no_ok_validations, no_not_ok_validations, percent_ok, percent_not_ok]

In [5]:
cols = ['CustomerID', 'Month', 'No_planned_visits', 'No_executed_visits', 'No_not_executed_visits', 
        'No_adhoc_visits', 'Day_maximum_difference', 'Day_maximum_sape', 'Day_minimum_sape', 'Day_validation',
        'No_validations', 'Mean_SAPE', 'Std_dev_SAPE', 'Mean_APE', 'Squared_error_SAPE_APE', 
        'Mean_FPE', 'Std_dev_FPE', 'No_ok_validations', 'No_not_ok_validations', 'Percent_ok', 'Percent_not_ok']

df_expert_vectors = pd.DataFrame(data=list(expert_vectors.values()), columns=cols)
df_expert_vectors = df_expert_vectors.sort_values(by=['CustomerID', 'Month'])
df_expert_vectors.to_csv('dataset/Feature_vectors_by_customerID_and_month.tsv', sep='\t', index=None)